In [ ]:
%matplotlib inline

# Diffusive Grids

This is just a simple implementation of the idea of 1D adaptive vertical coordinates (Burchard and Beckers, 2004). We take a random profile from WOA13 to give us a stratification, and explore the effect of the stratification, near-surface and background grid diffusivity coefficients.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
import gsw
from scipy.interpolate import interp1d
from scipy.linalg import solve_banded

Load WOA13 and grab the in-situ temperature and practical salinity, as well as coordinate variables.

In [ ]:
temp_d = Dataset('../convert_WOA13/raw/woa13_A5B2_t01_01v2.nc', 'r')
salt_d = Dataset('../convert_WOA13/raw/woa13_A5B2_s01_01v2.nc', 'r')

dep = temp_d.variables['depth'][:]
lat = temp_d.variables['lat'][:]
lon = temp_d.variables['lon'][:]

temp = temp_d.variables['t_an']
salt = salt_d.variables['s_an']

Just choose a nice full-depth (1500m, 57 levels) profile from the Denmark Strait. The observations are annoying because they include the surface and the very bottom, so we'll just use `dep` as our interface locations and average to get observation locations.

In [ ]:
lat_i = lat == 63.5
lon_i = (-39.5 <= lon) & (lon <= -22.5)

temp_c = temp[0,:,lat_i,lon_i].squeeze()
salt_c = salt[0,:,lat_i,lon_i].squeeze()

# average observations between original locations
dep_lay = (dep[1:]    + dep[:-1])    / 2
temp_c  = (temp_c[1:,:] + temp_c[:-1,:]) / 2
salt_c  = (salt_c[1:,:] + salt_c[:-1,:]) / 2

Use GSW (TEOS-10) to convert to absolute salinity and conservative temperature, then compute the density referenced to 2000m and the buoyancy frequency *on interfaces*.

In [ ]:
sa_c  = np.empty_like(salt_c)
ct_c  = np.empty_like(salt_c)
r_c   = np.empty_like(salt_c)
n2    = np.empty((salt_c.shape[0] - 1, salt_c.shape[1]))
# new interface depths
dep_c = np.empty_like(n2)

# calcluate for each column
for i in range(salt_c.shape[1]):
    sa_c[:,i]           = gsw.SA_from_SP(salt_c[:,i], dep_lay, lon[lon_i][i], 63.5)
    ct_c[:,i]           = gsw.CT_from_t(sa_c[:,i], temp_c[:,i], dep_lay)
    r_c[:,i]            = gsw.rho(sa_c[:,i], ct_c[:,i], 2000)
    n2[:,i], dep_c[:,i] = gsw.Nsquared(sa_c[:,i], ct_c[:,i], dep_lay)

Convert from buoyancy frequency to stratification by eliminating gravity and a weird decibar to pascal conversion.

In [ ]:
g = 9.7963
dz_r = np.maximum((n2 * 1e4) / g**2, 0)
k_n2_int = dz_r / (r_c[-1,:] - r_c[0,:])

Plot our $k^\text{grid}_{N^2}$ diffusivity, on interfaces. We need to interpolate it back to layers so that the diffusion equation applies to interfaces.

In [ ]:
plt.plot(k_n2_int);

Now we need to interpolate this from interfaces `dep_c` to layers `dep_lay`.

In [ ]:
k_n2 = np.empty_like(sa_c)

for i in range(k_n2.shape[1]):
    f         = interp1d(dep_c[:,i], k_n2_int[:,i], bounds_error=False, fill_value="extrapolate")
    k_n2[:,i] = f(dep_lay)

The discretised implicit diffusion equation is
$$ z_i^{n+1} = z_i^n + I^2 \Delta t^\text{grid}\left(k^\text{grid}_{i+1}\left(z_{i+1}^{n+1} - z_i^{n+1}\right) - k^\text{grid}_i\left(z_i^{n+1} - z_{i-1}^{n+1}\right)\right), $$
which just gives a tridiagonal system. $I$ is the number of layers.

Upper diagonal coefficients are $-I^2 \Delta t k^\text{grid}_{i+1}$, lower diagonal coefficients are similarly $-I^2 \Delta t k^\text{grid}_i$, and diagonal coefficients are simply
$$ 1 + I^2 \Delta t \left(k^\text{grid}_{i+1} + k^\text{grid}_i\right).$$

In [ ]:
dep_next = np.empty((dep_c.shape[0] + 2, dep_c.shape[1]))

for i in range(dep_next.shape[1]):
    # determine total grid coefficient from
    # buoyancy term k_n2
    # background term 1/D
    k_grid = (1500 * 0.7 * k_n2[:,i] + 0.3) / 3600

    dt = 300               # one hour restoring timescale?
    I = dep_c.shape[0] + 1     # number of layers
    A = np.zeros((3, I+1)) # diffusion system coefficients
    # add surface and bottom to interfaces
    dep_i = np.hstack([0, dep_c[:,i], 1500])

    A[0,2:]     = -dt * I**2 * k_grid[1:]
    A[2,:-2]    = -dt * I**2 * k_grid[:-1]
    A[1,[0,-1]] = 1 # boundary conditions
    A[1,1:-1]   = 1 + I**2 * dt * (k_grid[1:] + k_grid[:-1])

    # solve tridiagonal system
    dep_next[:,i] = solve_banded((1, 1), A, dep_i)

In [ ]:
plt.plot(dep_next[1:-1,:], '*')
plt.gca().invert_yaxis()